
# Trabalho 1: Diferenciação Automática com Grafos Computacionais

## Informações Gerais

- Data de Entrega: 29/06/2025
- Pontuação: 10 pontos (+4 pontos extras)
- O trabalho deve ser feito individualmente.
- A entrega do trabalho deve ser realizada via sistema testr.



## Especificação

⚠️ *Esta explicação assume que você leu e entendeu os slides sobre grafos computacionais.*

O trabalho consiste em implementar um sistema de diferenciação automática usando grafos computacionais e utilizar este sistema para resolver um conjunto de problemas.

Para isto, devem ser definidos um tipo Tensor para representar dados (similares aos arrays do numpy) e operações (e.g., soma, subtração, etc.) que geram tensores como saída. 

Sempre que uma operação é realizada, é armazenado no tensor de saída referências para os seus pais, isto é, os valores usados como entrada para a operação. 


### Imports

In [1]:
from typing import Optional, Union, Any
from collections.abc import Iterable
from abc import ABC, abstractmethod
import numbers

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

### Classe NameManager

A classe NameManager provê uma forma conveniente de dar nomes intuitivos para tensores que resultam de operações. A idéia é tornar mais fácil para o usuário das demais classes qual operação gerou qual tensor. Ela provê os seguintes métodos públicos: 

- reset(): reinicia o sistema de gestão de nomes.
- new(<basename>: str): retorna um nome único a partir do nome de base passado como argumento. 
  
Como indicado no exemplo abaixo da classe, a idéia geral é que uma sequência de operações é feita, os nomes dos tensores sejam os nomes das operações seguidos de um número. Se forem feitas 3 operações de soma e uma de multiplicação, seus tensores de saída terão os nomes "add:0", "add:1", "add:2" e "prod:0".

In [2]:

class NameManager:
    _counts = {}

    @staticmethod
    def reset():
        NameManager._counts = {}

    @staticmethod
    def _count(name):
        if name not in NameManager._counts:
            NameManager._counts[name] = 0
        count = NameManager._counts[name]
        return count

    @staticmethod
    def _inc_count(name):
        assert name in NameManager._counts, f'Name {name} is not registered.'
        NameManager._counts[name] += 1

    @staticmethod
    def new(name: str):
        count = NameManager._count(name)
        tensor_name = f"{name}:{count}"
        NameManager._inc_count(name)
        return tensor_name

# exemplo de uso
print(NameManager.new('add'))
print(NameManager.new('in'))
print(NameManager.new('add'))
print(NameManager.new('add'))
print(NameManager.new('in'))
print(NameManager.new('prod'))

NameManager.reset()

add:0
in:0
add:1
add:2
in:1
prod:0


### Classe Tensor

Deve ser criada uma classe `Tensor` representando um array multidimensional.

In [ ]:

class Tensor:
    """Classe representando um array multidimensional.

    Atributos:

    - _arr  (privado): dados internos do tensor como
        um array do numpy com 2 dimensões (ver Regras)

    - _parents (privado): lista de tensores que foram
        usados como argumento para a operação que gerou o
        tensor. Será vazia se o tensor foi inicializado com
        valores diretamente. Por exemplo, se o tensor foi
        resultado da operação a + b entre os tensores a e b,
        _parents = [a, b].

    - requires_grad (público): indica se devem ser
        calculados gradientes para o tensor ou não.

    - grad (público): Tensor representando o gradiente.

    """

    def __init__(self,
                 # Dados do tensor. Além dos tipos listados,
                 # arr também pode ser do tipo Tensor.
                 arr: Union[np.ndarray, list, numbers.Number, Any],
                 # Entradas da operacao que gerou o tensor.
                 # Deve ser uma lista de itens do tipo Tensor.
                 parents: list[Any] = [],
                 # se o tensor requer o calculo de gradientes ou nao
                 requires_grad: bool = True,
                 # nome do tensor
                 name: str = '',
                 # referência para um objeto do tipo Operation (ou
                 # subclasse) indicando qual operação gerou este
                 # tensor. Este objeto também possui um método
                 # para calcular a derivada da operação.
                 operation=None):
        """Construtor

        O construtor deve permitir a criacao de tensores das seguintes formas:

            # a partir de escalares
            x = Tensor(3)

            # a partir de listas
            x = Tensor([1,2,3])

            # a partir de arrays
            x = Tensor(np.array([1,2,3]))

            # a partir de outros tensores (construtor de copia)
            x = Tensor(Tensor(np.array([1,2,3])))

        Para isto, as seguintes regras devem ser obedecidas:

        - Se o argumento arr não for um array do numpy,
            ele deve ser convertido em um. Defina o dtype do
            array como float de forma a permitir que NÃO seja
            necessário passar constantes float como Tensor(3.0),
            mas possamos criar um tensor apenas com Tensor(3).

        - O atributo _arr deve ser uma matriz, isto é,
            ter 2 dimensões (ver Regras).

        - Se o argumento arr for um Tensor, ele deve ser
            copiado (cuidado com cópias por referência).

        - Se arr for um array do numpy com 1 dimensão,
            ele deve ser convertido em uma matriz coluna.

        - Se arr for um array do numpy com dimensão maior
            que 2, deve ser lançada uma exceção.

        - Tensores que não foram produzidos como resultado
            de uma operação não têm pais nem operação.
            Os nomes destes tensores devem seguir o formato in:3.
        """
        # TODO


    def zero_grad(self):
        """Reinicia o gradiente com zero"""
        # TODO

    def numpy(self):
        """Retorna o array interno"""
        # TODO

    def __repr__(self):
        """Permite visualizar os dados do tensor como string"""
        return f"Tensor({self._arr}, name={self._name}, shape={self._arr.shape})"

    def backward(self, my_grad=None):
        """Método usado tanto iniciar o processo de
        diferenciação automática, quanto por um filho
        para enviar o gradiente do pai. No primeiro
        caso, o argumento my_grad não será passado.
        """
        # TODO


### Interface de  Operações

A classe abaixo define a interface que as operações devem implementar. Ela não precisa ser modificada, mas pode, caso queira.

In [ ]:

class Op(ABC):
    @abstractmethod
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza a operação usando as entradas e
            retorna o tensor resultado. O método deve
            garantir que o atributo parents do tensor
            de saída seja uma lista de tensores."""

    @abstractmethod
    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna os gradientes dos pais em como tensores.

        Arguments:

        - back_grad: Derivada parcial em relação à saída
            da operação backpropagada pelo filho.

        - args: variaveis de entrada da operacao (pais)
            como tensores.

        - O nome dos tensores de gradiente devem ter o
            nome da operacao seguido de '_grad'.
        """


### Implementação das Operações

Operações devem herdar de `Op` e implementar os métodos `__call__` e `grad`.

Pelo menos as seguintes operações devem ser implementadas:



In [ ]:

class Add(Op):
    """Add(a, b): a + b"""
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza a operação usando os argumentos dados em args"""

    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""

# Instancia a classe. O objeto passa a poder ser usado como uma funcao
add = Add()

In [ ]:

class Sub(Op):
    """Sub(a, b): a - b"""
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza a operação usando os argumentos dados em args"""

    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""

# Instancia a classe. O objeto passa a poder ser usado como uma funcao
sub = Sub()

In [ ]:

class Prod(Op):
    """Prod(a, b): produto ponto a ponto de a e b ou produto escalar-tensor"""
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza a operação usando os argumentos dados em args"""

    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""

# Instancia a classe. O objeto passa a poder ser usado como uma funcao
prod = Prod()

In [ ]:

class Sin(Op):
    """seno element-wise"""
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza a operação usando os argumentos dados em args"""

    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""

# Instancia a classe. O objeto passa a poder ser usado como uma funcao
sin = Sin()

In [ ]:

class Cos(Op):
    """cosseno element-wise"""
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza a operação usando os argumentos dados em args"""

    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""


# Instancia a classe. O objeto passa a poder ser usado como uma funcao
cos = Cos()

In [ ]:

class Sum(Op):
    """Retorna a soma dos elementos do tensor"""
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza a operação usando os argumentos dados em args"""

    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""

# Instancia a classe. O objeto passa a poder ser usado como uma funcao
# ⚠️ vamos chamar de my_sum porque python ja possui uma funcao sum
my_sum = Sum()

In [ ]:

class Mean(Op):
    """Retorna a média dos elementos do tensor"""
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza a operação usando os argumentos dados em args"""

    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""

# Instancia a classe. O objeto passa a poder ser usado como uma funcao
mean = Mean()

In [ ]:

class Square(Op):
    """Eleva cada elemento ao quadrado"""
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza a operação usando os argumentos dados em args"""

    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""

# Instancia a classe. O objeto passa a poder ser usado como uma funcao
square = Square()

In [ ]:

class MatMul(Op):
    """MatMul(A, B): multiplicação de matrizes

    C = A @ B
    de/dA = de/dc @ B^T
    de/dB = A^T @ de/dc

    """

    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza a operação usando os argumentos dados em args"""

    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""

# Instancia a classe. O objeto passa a poder ser usado como uma funcao
matmul = MatMul()

In [ ]:

class Exp(Op):
    """Exponenciação element-wise"""
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza a operação usando os argumentos dados em args"""

    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""

# Instancia a classe. O objeto passa a poder ser usado como uma funcao
exp = Exp()

In [ ]:

class ReLU(Op):
    """ReLU element-wise"""
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza a operação usando os argumentos dados em args"""

    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""

# Instancia a classe. O objeto passa a poder ser usado como uma funcao
relu = ReLU()

In [ ]:

class Sigmoid(Op):
    """Sigmoid element-wise"""
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza a operação usando os argumentos dados em args"""

    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""

# Instancia a classe. O objeto passa a poder ser usado como uma funcao
sigmoid = Sigmoid()

In [ ]:

class Tanh(Op):
    """Tanh element-wise"""
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza a operação usando os argumentos dados em args"""

    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""

# Instancia a classe. O objeto passa a poder ser usado como uma funcao
tanh = Tanh()

In [ ]:

class Softmax(Op):
    """Softmax de um array de valores. Lembre-se que cada elemento do array influencia o resultado da função para todos os demais elementos."""
    def __call__(self, *args, **kwargs) -> Tensor:
        """Realiza a operação usando os argumentos dados em args"""

    def grad(self, back_grad: Tensor, *args, **kwargs) -> list[Tensor]:
        """Retorna a lista de derivadas parciais em relação aos pais (passados em args)"""

# Instancia a classe. O objeto passa a poder ser usado como uma funcao
softmax = Softmax()


### ‼️ Regras e Pontos de Atenção‼️

- Vamos fazer a hipótese simplificadora que Tensores devem ser sempre matrizes. Por exemplo, o escalar 2 deve ser armazado em `_arr` como a matriz `[[2]]`. De forma similar, a lista `[1, 2, 3]` deve ser armazenada em `_arr` como em uma matriz coluna.

- Devem ser realizados `asserts` nas operações para garantir que os shapes dos operandos fazem sentido. Esta verificação também deve ser feita depois das operações que manipulam gradientes de tensores.

- Devem ser respeitados os nomes dos atributos, métodos e classes para viabilizar os testes automáticos.

- Gradientes devem ser calculados usando uma passada pelo grafo computacional.

- Os gradientes devem ser somados e não substituídos nas chamadas de  backward. Isto vai permitir que os gradientes sejam acumulados entre amostras do dataset e que os resultados sejam corretos mesmo em caso de ramificações e junções no grafo computacional.

- Lembre-se de zerar os gradientes após cada passo de gradient descent (atualização dos parâmetros).


## Testes Básicos

Estes testes avaliam se a derivada da função está sendo calculada corretamente, mas em muitos casos **não** avaliam se os gradientes backpropagados estão sendo incorporados corretamente. Esta avaliação será feita nos problemas da próxima seção.

Operador de Soma

In [ ]:
# add

a = Tensor([1.0, 2.0, 3.0])
b = Tensor([4.0, 5.0, 6.0])
c = add(a, b)
d = add(c, 3.0)
d.backward()

# esperado: matrizes coluna contendo 1
print(a.grad)
print(b.grad)


Tensor([[1.]
 [1.]
 [1.]], name=in_grad, shape=(3, 1))
Tensor([[1.]
 [1.]
 [1.]], name=in_grad, shape=(3, 1))


Operador de Subtração

In [ ]:
# sub

a = Tensor([1.0, 2.0, 3.0])
b = Tensor([4.0, 5.0, 6.0])
c = sub(a, b)
d = sub(c, 3.0)
d.backward()

# esperado: matrizes coluna contendo 1 e -1
print(a.grad)
print(b.grad)


Tensor([[1.]
 [1.]
 [1.]], name=in_grad, shape=(3, 1))
Tensor([[-1.]
 [-1.]
 [-1.]], name=in_grad, shape=(3, 1))


Operador de Produto

In [ ]:
# prod

a = Tensor([1.0, 2.0, 3.0])
b = Tensor([4.0, 5.0, 6.0])
c = prod(a, b)
d = prod(c, 3.0)
d.backward()

# esperado: [12, 15, 18]^T
print(a.grad)
# esperado: [3, 6, 9]^T
print(b.grad)


Tensor([[12.]
 [15.]
 [18.]], name=in_grad, shape=(3, 1))
Tensor([[3.]
 [6.]
 [9.]], name=in_grad, shape=(3, 1))


Operadores trigonométricos

In [ ]:
# sin e cos

a = Tensor([np.pi, 0, np.pi/2])
b = sin(a)
c = cos(a)
d = my_sum(add(b, c))
d.backward()

# esperado: [-1, 1, -1]^T
print(a.grad)

Tensor([[-1.]
 [ 1.]
 [-1.]], name=in_grad, shape=(3, 1))


In [ ]:
# Sum

a = Tensor([3.0, 1.0, 0.0, 2.0])
b = add(prod(a, 3.0), a)
c = my_sum(b)
c.backward()

# esperado: [4, 4, 4, 4]^T
print(a.grad)


Tensor([[4.]
 [4.]
 [4.]
 [4.]], name=in_grad, shape=(4, 1))


In [ ]:
# Mean

a = Tensor([3.0, 1.0, 0.0, 2.0])
b = mean(a)
b.backward()

# esperado: [0.25, 0.25, 0.25, 0.25]^T
print(a.grad)


Tensor([[0.25]
 [0.25]
 [0.25]
 [0.25]], name=in_grad, shape=(4, 1))


In [ ]:
# Square

a = Tensor([3.0, 1.0, 0.0, 2.0])
b = square(a)

# esperado: [9, 1, 0, 4]^T
print(b)

b.backward()

# esperado: [6, 2, 0, 4]
print(a.grad)

Tensor([[9.]
 [1.]
 [0.]
 [4.]], name=square:0, shape=(4, 1))
Tensor([[6.]
 [2.]
 [0.]
 [4.]], name=in_grad, shape=(4, 1))


In [ ]:
# matmul

W = Tensor([
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0],
    [7.0, 8.0, 9.0]
])

v = Tensor([1.0, 2.0, 3.0])

z = matmul(W, v)

# esperado: [14, 32, 50]^T
print(z)

z.backward()

# esperado:
# [1, 2, 3]
# [1, 2, 3]
# [1, 2, 3]
print(W.grad)

# esperado: [12, 15, 18]^T
print(v.grad)


Tensor([[14.]
 [32.]
 [50.]], name=matmul:0, shape=(3, 1))
Tensor([[1. 2. 3.]
 [1. 2. 3.]
 [1. 2. 3.]], name=in_grad, shape=(3, 3))
Tensor([[12.]
 [15.]
 [18.]], name=in_grad, shape=(3, 1))


In [ ]:
# Exp

v = Tensor([1.0, 2.0, 3.0])
w = exp(v)

# esperado: [2.718..., 7.389..., 20.085...]^T
print(w)

w.backward()

# esperado: [2.718..., 7.389..., 20.085...]^T
print(v.grad)

Tensor([[ 2.71828183]
 [ 7.3890561 ]
 [20.08553692]], name=exp:0, shape=(3, 1))
Tensor([[ 2.71828183]
 [ 7.3890561 ]
 [20.08553692]], name=in_grad, shape=(3, 1))


In [17]:
# Relu

v = Tensor([-1.0, 0.0, 1.0, 3.0])
w = relu(v)

# esperado: [0, 0, 1, 3]^T
print(w)

w.backward()

# esperado: [0, 0, 1, 1]^T
print(v.grad)

Tensor([[0.]
 [0.]
 [1.]
 [3.]], name=relu:0, shape=(4, 1))
Tensor([[0.]
 [0.]
 [1.]
 [1.]], name=in_grad, shape=(4, 1))


In [18]:
# Sigmoid

v = Tensor([-1.0, 0.0, 1.0, 3.0])
w = sigmoid(v)

# esperado: [0.268.., 0.5, 0.731.., 0.952..]^T
print(w)

w.backward()

# esperado: [0.196..., 0.25, 0.196..., 0.045...]^T
print(v.grad)

Tensor([[0.26894142]
 [0.5       ]
 [0.73105858]
 [0.95257413]], name=sigmoid:0, shape=(4, 1))
Tensor([[0.19661193]
 [0.25      ]
 [0.19661193]
 [0.04517666]], name=in_grad, shape=(4, 1))


In [19]:
# Tanh

v = Tensor([-1.0, 0.0, 1.0, 3.0])
w = tanh(v)

# esperado: [[-0.76159416, 0., 0.76159416, 0.99505475]^T
print(w)

w.backward()

# esperado: [0.41997434, 1., 0.41997434, 0.00986604]^T
print(v.grad)

Tensor([[-0.76159416]
 [ 0.        ]
 [ 0.76159416]
 [ 0.99505475]], name=tanh:0, shape=(4, 1))
Tensor([[0.41997434]
 [1.        ]
 [0.41997434]
 [0.00986604]], name=in_grad, shape=(4, 1))


In [ ]:
# Softmax

x = Tensor([-3.1, 0.5, 1.0, 2.0])
y = softmax(x)

# esperado: [0.00381737, 0.13970902, 0.23034123, 0.62613238]^T
print(y)

# como exemplo, calcula o MSE para um target vector
diff = sub(y, [1, 0, 0, 0])
sq = square(diff)
a = mean(sq)

# esperado: 0.36424932
print("MSE:", a)

a.backward()

# esperado: [-0.00278095, -0.02243068, -0.02654377, 0.05175539]^T
print(x.grad)



Tensor([[0.00381737]
 [0.13970902]
 [0.23034123]
 [0.62613238]], name=softmax:6, shape=(4, 1))
MSE: Tensor([[0.36424932]], name=mean:7, shape=(1, 1))
Tensor([[-0.00278095]
 [-0.02243068]
 [-0.02654377]
 [ 0.05175539]], name=in_grad, shape=(4, 1))


## Pontos Extras

### Tarefas

- **+2 pontos**: Utilizar sobrecarga de operadores para permitir que todas as operações disponíveis aos arrays do numpy possam ser realizadas com tensores, incluindo operações que envolvam broadcasting.
  - Por exemplo, assumindo que a e b são tensores possivelmente com dimensões diferentes, devem ser possível realizar as operações a + 2, a * b, a @ b, a.max(), a.sum(axis=0).
  - Para realizar esta atividade, os atributos da classe Tensor podem ser completamente modificados, mas deve ser provido um método backward para iniciar o backpropagation.
  - Naturalmente, a regra de que tensores devem ser matrizes deve ser desconsiderada neste caso.

- **+1 ponto**: Atualizar as classes para permitir derivadas de mais alta ordem (derivadas segundas, etc.).

- **+1 ponto**: Entregar uma versão adicional do trabalho completo usando C/C++ e com foco em minimizar o tempo para realização das operações. Os casos de teste do sistema Testr também deverão ser replicados utilizando esta linguagem.

### Regras

- Só serão elegíveis para receber pontos extras os alunos que cumprirem 100% dos requisitos da parte principal do trabalho.

- Para receber os pontos extras, deverá ser agendado um horário para uma entrevista individual que abordará tanto os códigos-fonte relativos aos pontos extras quanto à parte principal do trabalho (pode acontecer redução da pontuação da parte principal do trabalho).

- Receberá os pontos extras quem responder corretamente às perguntas da entrevista. Não será atribuída pontuação parcial aos pontos extras.

## Referências

### Principais

- [Build your own pytorch](https://www.peterholderrieth.com/blog/2023/Build-Your-Own-Pytorch-1-Computation-Graphs/)
- [Build your own Pytorch - 2: Backpropagation](https://www.peterholderrieth.com/blog/2023/Build-Your-Own-Pytorch-2-Autograd/)
- [Build your own PyTorch - 3: Training a Neural Network with self-made AD software](https://www.peterholderrieth.com/blog/2023/Build-Your-Own-Pytorch-3-Build-Classifier/)
- [Pytorch: A Gentle Introduction to torch.autograd](https://docs.pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html)
- [Automatic Differentiation with torch.autograd](https://docs.pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html)

### Secundárias

- [Tom Roth: Building a computational graph: part 1](https://tomroth.dev/compgraph1/)
- [Tom Roth: Building a computational graph: part 2](https://tomroth.dev/compgraph2/)
- [Tom Roth: Building a computational graph: part 3](https://tomroth.dev/compgraph3/)
- [Roger Grosse (Toronto) class on Automatic Differentiation](https://www.cs.toronto.edu/~rgrosse/courses/csc321_2018/slides/lec10.pdf)
- [Computational graphs and gradient flows](https://simple-english-machine-learning.readthedocs.io/en/latest/neural-networks/computational-graphs.html)
- [Colah Visual Blog: Backprop](https://colah.github.io/posts/2015-08-Backprop/)
- [Towards Data Science: Automatic Differentiation (AutoDiff): A Brief Intro with Examples](https://towardsdatascience.com/automatic-differentiation-autodiff-a-brief-intro-with-examples-3f3d257ffe3b/)
- [A Hands-on Introduction to Automatic Differentiation - Part 1](https://mostafa-samir.github.io/auto-diff-pt1/)
- [Build Your own Deep Learning Framework - A Hands-on Introduction to Automatic Differentiation - Part 2](https://mostafa-samir.github.io/auto-diff-pt1/)
